# Исключения

Решения необходимо отправить через систему [Яндекс.Контест](https://contest.yandex.ru/contest/69679/enter/?retPage=).

## Задача 1. Совместимость с API

На прошлом занятии мы с вами изучили интерфейсы. Помимо сигнатур методов частью интерфейса также являются исключения, которые возбуждаются данными методами. На уровне реализации методы того или иного объекта, разделяющего наш интерфейс, могут использовать сторонние библиотеки. Сторонний код может приводить к появлению исключений, несовместимых с исключениями нашего интерфейса, поэтому их необходимо обрабатывать и преобразовывать в пределах нашего метода, чтобы сохранять соответствие интерфейсу.

Проблему с обработкой можно решить двумя способами: написать обработчики в самом методе или написать декоратор общего назначения для обработки исключений. Первый путь является более гибким с точки зрения обработки исключений, т.к. мы можем определять четкую последовательность действий в обработчики каждого исключения. Однако с точки зрения поддержки этот путь является менее гибким и может привести к большому количеству однообразного кода. Собственно, эти недостатки позволяет решить второй способ.

Итак, ваша задача - реализовать параметризованный декоратор для преобразования исключений. В качестве параметра декоратора выступает словарь, определяющий правило преобразования исключений. Если возбужденного исключения не окажется в переданном словаре, необходимо перевозбудить это исключение без изменений.

**Вход**:
- На вход параметризованному декоратору передается словарь вида `{Exception: Exception}`. Ключам соответствуют типы исключений, которые мы хотим преобразовывать, значениям - типы исключений, к которым мы хотим преобразовывать. Т.е. если нам был передан словарь `{ValueError: MyException}`, то в случае возникновения `ValueError` в продекорированной функции в процессе выполнения, мы должны будем возбудить `MyException`. При этом трейсбек исходного исключения должен быть удален. Если же в продикорированной функции возникнет исключение, которого нет в словаре, например, `KeyError`, мы должны будем возбудить его без изменений.

**Выход**:
- Продикорированная функция, обладающая API-совместимыми исключениями.

**Замечание**:  
Предполагаем, что декорируемые функции могу возбуждать исключительно `Exception` и ее подклассы.

**Решение**:

In [1]:
from functools import wraps
from typing import Callable, TypeVar

In [2]:
T = TypeVar("T")

def api_computable_exceptions(
    exception_mapping: dict[type[Exception], type[Exception]]
) -> Callable[[T], T]:
    # ваш код
    pass

**Проверка**:

In [3]:
class UnsupportedValueError(Exception):
    pass


class NonExistedKeyError(Exception):
    pass

In [ ]:
exception_mapping = {
    ValueError: UnsupportedValueError,
    KeyError: NonExistedKeyError,
}


@api_computable_exceptions(exception_mapping)
def raise_value_error() -> None:
    raise ValueError


@api_computable_exceptions(exception_mapping)
def raise_key_error() -> None:
    raise KeyError


@api_computable_exceptions(exception_mapping)
def raise_exception() -> None:
    raise Exception

In [ ]:
try:
    raise_value_error()
    assert False
except UnsupportedValueError:
    pass

try:
    raise_key_error()
    assert False
except NonExistedKeyError:
    pass

try:
    raise_exception()
except Exception as exc:
    assert isinstance(exc, Exception)

## Задача 2. Точность

Напомним, что встроенный тип данных для работы с числами с плавающей точкой `float` в Python обладает ограниченной точностью из-за особенностей хранения чисел с плавающей точкой в памяти компьютера. Более того, из-за этих особенностей вы часто можете получить неожиданные результаты. К числу неожиданных результатов, например, относится следующая сумма:

```console
>>> 1.1 + 2.2
3.3000000000000003
```

Для того, чтобы сделать работу с числами с плавающей точкой в Python более удобной, интуитивной и ожидаемой, был добавлен модуль `decimal`. Этот модуль позволяет оперировать объектами `Decimal` и получать ожидаемые результаты:

```python
import decimal
result = decimal.Decimal("1.1") + decimal.Decimal("2.2")
print(result)
# 3.3
```

Подробнее ознакомиться с возможностями данного модуля можно в [официальной документации](https://docs.python.org/3/library/decimal.html). Помимо прочего, модуль `decimal` позволяет вам явно задать точность, с которой будет происходить вычисление результата при помощи объектов `Decimal`. Например:

```python
from decimal import Decimal, getcontext
getcontext().prec = 3
print(Decimal("1") / Decimal("3"))
# 0.333
getcontext().prec = 20
print(Decimal("1") / Decimal("3"))
# 0.33333333333333333333
```

Однако, как быть если нам необходимо изменить точность для проведения конкретных вычислений, а затем вернуться к исходному значению? Самый простой способ: запомнить текущую точность, провести нужные вычисления, а затем, вернуть исходную точность:

```python
from decimal import Decimal, getcontext

print(Decimal("1") / Decimal("3"))
prec_curr = getcontext().prec
print(f"{prec_curr = }")
# 0.33333333333333333333
# prec_curr = 20

getcontext().prec = 3
print(Decimal("1") / Decimal("3"))
print(f"prec = {getcontext().prec}")
# 0.333
# prec = 3

getcontext().prec = prec_curr
print(Decimal("1") / Decimal("3"))
print(f"{prec_curr = }")
# 0.33333333333333333333
# prec_curr = 20
```

Однако такой подход не застрахован от ошибок. Если в момент проведения вычислений с измененной точностью произойдет какое-либо исключение, исходная точность не будет восстановлена. Чтобы решить эту проблему, вам необходимо реализовать контекстный менеджер для изменения точности вычислений:

```python
from decimal import Decimal, getcontext

print(f"prec = {getcontext().prec}")
# prec = 20

with Precision(3):
    print(Decimal("1") / Decimal("3"))
    print(f"prec = {getcontext().prec}")
    # 0.333
    # prec = 3

print(f"prec = {getcontext().prec}")
# prec = 20
```

**Вход**:
- На вход конструктору контекстного менеджера `Precision` подается целое число - требуемая точность. Если пользователем было передано число с плавающей точкой, необходимо округлить его до ближайшего целого числа. Если переданный объект не поддерживает округления - необходимо возбудить `TypeError`. Если полученное после округления число - это число, меньшее единицы, необходимо возбудить `ValueError`.

**Решение**:

In [15]:
from decimal import Decimal, getcontext
from types import TracebackType
from typing import Optional

In [16]:
class Precision:
    def __init__(self, precision: int) -> None:
        # ваш код
        pass

In [ ]:
precision = 5

with Precision(precision):
    assert getcontext().prec == precision
    print(Decimal("1") / Decimal("3"))

## Задача 3. Пункт назначения

В объекте `stdout` встроенного модуля `sys` хранится объект, в который интерпретатор записывает результат выполнения функции `print()`, а также промпты, которые печатает функция `input()`. Заменив объект `sys.stdout` мы можем изменить буфер, в который будут писаться наши сообщения. Например, мы можем изменить пункт назначения сообщений со стандартного потока вывода на файл следующим образом:

```python
import sys

file = open("out.txt", "w")
stdout = sys.stdout
sys.stdout = file
print("Hello!")
sys.stdout = stdout
file.close()
```

В данном примере, до тех пор, пока `sys.stdout` не будет возвращено изначальное значение, все сообщения будут записываться в файл `out.txt`. Однако, очевидно, данный подход обладает существенными недостатками. Если в процессе записи произойдет какое-либо исключение, мы не вернем `sys.stdout` первоначальное значение, и последующие результаты выполнения функций `print()` продолжат писаться в `out.txt`. Более того, если случится исключение, мы не закроем файловый дескриптор файла `out.txt`, а, следовательно, потеряем часть данных и столкнемся с проблемами похуже, в виде утечек памяти.

Чтобы этого избежать, логичнее было бы разработать контекстный менеджер, который бы инкапсулировал бы логику работы с файлами и логику с заменой объекта `sys.stdout`, а также обеспечивал бы безопасную работу с ресурсами:

```python
with FileOut("test.txt") as file_manager:
    print(
        "Hello, World!",
        "This text must be printed into file",
        sep="\n",
    )
```

**Вход**:
- На вход конструктору контекстного менеджера `FileOut` передаются два объекта: `path_to_file`, `mode`. `path_to_file` - путь до файла, в который будет перенаправлен вывод. `mode` - опция, с которой будет открыт файл. У опций есть два значения: `"w"` - перезапись содержимого файла, `"a"` - запись в конец файла. Параметр `mode` является необязательным и имеет значение по умолчанию `"w"`. Если в параметр `mode` будут переданы значения отличные от допустимых, необходимо возбудить `ValueError`.

**Решение**:

In [18]:
import io
import sys

from enum import Enum
from types import TracebackType
from typing import Union, Optional, Self

In [19]:
class FileOutModes(Enum):
    APPEND = "a"
    REWRITE = "w"


class FileOut:
    def __init__(
        self,
        path_to_file: str,
        mode: Union[str, FileOutModes] = FileOutModes.REWRITE,
    ) -> None:
        # ваш код
        pass

    @property
    def mode(self) -> FileOutModes:
        # ваш код
        pass

    @mode.setter
    def mode(self, mode_new: Union[str, FileOutModes]) -> None:
        # ваш код
        pass

**Проверка**:

In [ ]:
with FileOut("test.txt") as file_manager:
    print(
        "Hello, World!",
        "This text must be printed into file",
        sep="\n",
    )

print("This text must be printed into stdout")

In [ ]:
file_manager.mode = "a"

with file_manager:
    print("Append more text!")

In [ ]:
try:
    file_manager.mode = "rewrite"

except ValueError:
    pass

else:
    assert False